In [69]:
import os
import pandas as pd
import numpy as np

from tqdm import tqdm
tqdm.pandas()

In [70]:
path = '../Data/Preprocessed/사용자분석.csv'

df_origin = pd.read_csv(path)
df_origin['날짜'] = pd.to_datetime(df_origin['날짜'], format = '%Y%m%d')
df_origin['가맹점 사업자 번호'] = df_origin['가맹점 사업자 번호'].apply(lambda x : str(x).zfill(10))

In [71]:
path = '../Data/Preprocessed/순환분석_main.csv'

df_franchise_main = pd.read_csv(path)
df_franchise_main['날짜'] = pd.to_datetime(df_franchise_main['날짜'], format = '%Y%m%d')
df_franchise_main['연도'] = df_franchise_main.apply(lambda x : str(x['날짜'].year) + '년', axis = 1)
df_franchise_main['사용자 사업자 번호'] = df_franchise_main['사용자 사업자 번호'].apply(lambda x : str(x).zfill(10))
df_franchise_main['가맹점 사업자 번호'] = df_franchise_main['가맹점 사업자 번호'].apply(lambda x : str(x).zfill(10))

In [72]:
path = '../Data/Origin/가맹점분류(대분류위주 분석)_231118.xlsx'

df_category = pd.read_excel(path)
df_category = df_category.rename(columns = {'사업자번호' : '가맹점 사업자 번호'})
df_category['가맹점 사업자 번호'] = df_category['가맹점 사업자 번호'].apply(lambda x : str(x).zfill(10))

In [73]:
agg_df = df_origin.groupby(['가맹점 사업자 번호', '연도'])['사용금액'].agg(['sum', 'mean'])

agg_df.columns = ['총 매출', '1회당 평균사용금액']
agg_df = agg_df.reset_index()

agg_df['1회당 평균사용금액'] = agg_df['1회당 평균사용금액'].apply(lambda x : round(x))

In [74]:
agg_franchise1 = df_franchise_main.groupby(['가맹점 사업자 번호', '연도'])['사용금액'].sum()
agg_franchise1.name = '순환소비 매출'
agg_franchise1 = agg_franchise1.reset_index()

agg_franchise2 = df_franchise_main.groupby(['사용자 사업자 번호', '연도'])['사용금액'].sum()
agg_franchise2.name = '순환소비 지출'
agg_franchise2 = agg_franchise2.reset_index()
agg_franchise2 = agg_franchise2.rename(columns = {'사용자 사업자 번호' : '가맹점 사업자 번호'})

agg_franchise1 = agg_franchise1.loc[~(agg_franchise1['가맹점 사업자 번호'].str.contains('삭제된가맹점'))]
agg_franchise2 = agg_franchise2.loc[~(agg_franchise2['가맹점 사업자 번호'].str.contains('삭제된가맹점'))]

agg_franchise = agg_franchise1.merge(agg_franchise2, on = ['가맹점 사업자 번호', '연도'], how = 'outer')
agg_franchise = agg_franchise.fillna(0)

In [75]:
agg_df = agg_df.merge(agg_franchise, on = ['가맹점 사업자 번호', '연도'], how = 'outer')
agg_df = agg_df.fillna(0)
agg_df['순환소비 매출 비율'] = round(agg_df['순환소비 매출'] / agg_df['총 매출'] * 100, 1)
agg_df['순환소비 지출 비율'] = round(agg_df['순환소비 지출'] / agg_df['총 매출'] * 100, 1)

In [76]:
agg_df

,가맹점 사업자 번호,연도,총 매출,1회당 평균사용금액,순환소비 매출,순환소비 지출,순환소비 매출 비율,순환소비 지출 비율
0,0011223344,2020년,465000.0,232500.0,0.0,465000.0,0.0,100.0
1,0011223344,2022년,880000.0,110000.0,0.0,880000.0,0.0,100.0
2,0123453155,2020년,1657123.0,276187.0,550000.0,596123.0,33.2,36.0
3,0418371882,2022년,2022500.0,87935.0,0.0,1981700.0,0.0,98.0
4,0418371882,2023년,5062000.0,105458.0,0.0,3725000.0,0.0,73.6
...,...,...,...,...,...,...,...,...
9828,8891102334,2021년,0.0,0.0,0.0,6294100.0,NaN,inf
9829,8891102334,2022년,0.0,0.0,0.0,3592900.0,NaN,inf
9830,8940802146,2020년,0.0,0.0,0.0,7765000.0,NaN,inf
9831,8940802146,2021년,0.0,0.0,0.0,3733750.0,NaN,inf


In [77]:
agg_df = agg_df.merge(df_category, on = ['가맹점 사업자 번호'], how = 'left')

In [79]:
# agg_df.to_csv('../Data/Preprocessed/가맹점인센티브분석.csv', index = False, encoding = 'utf-8-sig', escapechar = '\\')